In [47]:
# Cloud project id.
PROJECT_ID= !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
GCP_PROJECT = PROJECT_ID
VERTEX_API_LOCATION = LOCATION = REGION = 'us-central1'

print(GCP_PROJECT)
print(PROJECT_ID)

BUCKET_NAME = 'my-project-0004-bucket' # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output with gs:// prefix.
BUCKET_URI = "gs://my-project-0004-bucket"  # @param {type:"string"}

! gcloud config set project $PROJECT_ID

import os

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")

# The service account looks like:
# '@.iam.gserviceaccount.com'
# Please go to https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# and create service account with `Vertex AI User` and `Storage Object Admin` roles.
# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = "255766800726-compute@developer.gserviceaccount.com"  # @param {type:"string"}

# The pre-built docker images
TRAIN_DOCKER_URI = "us-central1-docker.pkg.dev/cloud-nas-260507/pytorch/transformers_train:latest"
SERVE_DOCKER_URI = "us-central1-docker.pkg.dev/cloud-nas-260507/pytorch/transformers_serve:latest"

# The serving port.
SERVE_PORT = 8081

my-project-0004-346516
my-project-0004-346516
Updated property [core/project].


## Define utility functions


In [48]:
import base64
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def download_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert("RGB")

def image_to_base64(image):
    buffer = BytesIO()
    image.save(buffer, format="JPEG")
    image_str = base64.b64encode(buffer.getvalue()).decode('utf-8')
    return image_str

def base64_to_image(image_str):
    image = Image.open(BytesIO(base64.b64decode(image_str)))
    return image

def image_grid(imgs, rows=2, cols=2):
    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h)) 
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

def draw_image_with_boxes(image, boxes):
    fig, ax = plt.subplots()
    ax.imshow(image)
    for box in boxes:
        x, y = box['xmin'], box['ymin']
        width, height = box['xmax'] - x, box['ymax'] - y
        rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor='yellow', facecolor='none')
        ax.add_patch(rect)
    plt.axis('off')
    plt.show()

In [49]:
VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20231002_0916_RC00"
MIXTRAL_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-mixtral-serve:20231214"

SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-diffusers-serve-opt:20240223_1230_RC00"


In [50]:
# [Optional] Run serving jobs locally
# Build and push serving docker image
# NOTE: Users won't build docker images on their own, they will use our pre-built docker images directly.

# !docker pull {VLLM_DOCKER_URI}
# !docker build -f serve.Dockerfile . -t {SERVE_DOCKER_URI}
# !docker push {SERVE_DOCKER_URI}

In [51]:
# !docker inspect {VLLM_DOCKER_URI} 

In [52]:


LOCAL_SERVE_URL = f"http://localhost:{SERVE_PORT}/predictions/transformers_serving"

### Zero-shot image-classification


In [53]:
# # Run the serving container in a separate shell.
# !nvidia-docker run -t --rm \
# -p {SERVE_PORT}:{SERVE_PORT} \
# -e NVIDIA_DISABLE_REQUIRE=1 \
# -e TASK=zero-shot-image-classification \
# -e MODEL_ID=openai/clip-vit-base-patch32 \
# -e CUDA_VISIBLE_DEVICES=0 \
# {VLLM_DOCKER_URI}

In [54]:
# VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20231002_0916_RC00"

# !nvidia-docker run -t --rm \
# -p 8081:8081 \
# -e NVIDIA_DISABLE_REQUIRE=1 \
# -e TASK=zero-shot-image-classification \
# -e MODEL_ID=aisingapore/sea-lion-7b \
# -e CUDA_VISIBLE_DEVICES=0 \
#  "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20231002_0916_RC00"


# Run the serving container in a separate shell.
# !nvidia-docker run -t --rm \
# -p {SERVE_PORT}:{SERVE_PORT} \
# -e NVIDIA_DISABLE_REQUIRE=1 \
# -e TASK=feature-embedding \
# -e MODEL_ID=openai/clip-vit-base-patch32 \
# {SERVE_DOCKER_URI}

In [55]:
instances = [
    {
        "prompt": "My favourite condiment is",
        "n": 1,
        "max_tokens": 200,
        "temperature": 1.0,
        "top_p": 1.0,
        "top_k": 10,
    },
]

In [56]:
LOCAL_SERVE_URL

'http://localhost:8081/predictions/transformers_serving'

In [57]:
import json

# image1 = download_image("http://images.cocodataset.org/val2017/000000039769.jpg")
# image2 = download_image("http://images.cocodataset.org/val2017/000000000285.jpg")
# grid = image_grid([image1, image2], 1, 2)
# display(grid)

payload = json.dumps({
  'instances': instances
})
r = requests.post(LOCAL_SERVE_URL, data=payload, headers={'content-type': 'application/json', 'Accept-Charset': 'UTF-8'})
print('r',r)
preds = r.json()['predictions']
print(preds)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
import json
import numpy as np

image = download_image("http://images.cocodataset.org/val2017/000000039769.jpg")
display(image)

payload = json.dumps({
  'instances': [
      {'image': image_to_base64(image), 'text': 'two cats'},
  ]
})
r = requests.post(LOCAL_SERVE_URL, data=payload, headers={'content-type': 'application/json', 'Accept-Charset': 'UTF-8'})
print('r',r)
preds = r.json()['predictions']
image_features = np.array(preds[0]['image_features'])
text_features = np.array(preds[0]['text_features'])
print(image_features.shape)
print(text_features.shape)

In [ ]:
!docker ps

In [ ]:
!docker stop $(docker ps -q)


In [ ]:
nvidia-docker run -t --rm \
-p 8082:8082 \
-e NVIDIA_DISABLE_REQUIRE=1 \
-e TASK=image-to-text \
-e MODEL_ID=nlpconnect/vit-gpt2-image-captioning \
us-central1-docker.pkg.dev/cloud-nas-260507/pytorch/transformers_serve:latest